In [ ]:
import pandas as pd
import numpy as np
import warnings
import os
from functools import reduce
warnings.filterwarnings(action='ignore')

import seaborn as sns
import matplotlib
sns.set_palette("pastel")
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
%matplotlib inline

sns.set(rc = {'figure.figsize':(15,7)})

import platform
your_os = platform.system()
if your_os == 'Linux':
    rc('font', family='NanumGothic')
elif your_os == 'Windows':
    ttf = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=ttf).get_name()
    rc('font', family=font_name)
elif your_os == 'Darwin':
    rc('font', family='AppleGothic')
rc('axes', unicode_minus=False)

# 외국인 유동인구
<br>
기간 : 2018.1.1 ~ 6.30

- base_date : 기준일자
- time : 시간
- nationality : 외국인 국적코드
- city	시 구분
- emd_cd : 행정동 코드
- emd_nm : 행정동명
- resd_pop_cnt : 거주인구
- work_pop_cnt : 근무인구
- visit_pop_cnt : 방문인구
- resd : 거주지역 행정구역

* 거주인구 : 1~24시 해당 시간 정각 측정 인구(거주지에 머문 시간(분)/60분)
* 근무인구 : 1~24시 해당 시간 정각 측정 인구(근무지에 머문 시간(분)/60분)
* 방문인구 : 1~24시 해당 시간 정각 측정 인구(거주지/근무지 외 지역에 머문 시간(분)/60분)

# Data Read

In [ ]:
#pd.read_csv('01_제공데이터/02-1_내국인유동인구_KOREAN.csv',low_memory=False)

frgn = pd.read_pickle('01_제공데이터/02-2_장기체류 외국인 유동인구_LONG_TERM_FRGN.pickle')

In [ ]:
frgn['base_date'] = pd.to_datetime(frgn['base_date'])

In [ ]:
def get_season(x):
    season = ''
    if (x == 1) or (x == 2) or (x == 12):
        season = 'winter'
    elif (x == 3) or (x == 4) or (x == 5):
        season = 'spring'
    elif (x == 6) or (x == 7) or (x == 8):
        season = 'summer'
    elif (x == 9) or (x == 10) or (x == 11):
        season = 'fall'
    else:
        season = ''
    return season

In [ ]:
frgn['month'] = frgn['base_date'].dt.month
frgn['season'] = frgn['month'].apply(lambda x: get_season(x)).astype('category')

In [ ]:
frgn['season'] = pd.Categorical(frgn['season'],['spring','summer','fall','winter'])

In [ ]:
nation_sort = frgn['nationality'].value_counts().index[:] # 인구 비중이 큰 국적부터 정렬

In [ ]:
frgn.head()

In [ ]:
frgn.info()

# Data preview(Total)

In [ ]:
# na값 확인
frgn.isna().sum()

In [ ]:
frgn['city'].value_counts()

국적 비중 : 중국 > 베트남 > 미국 > 인도 > 기타 > 네팔 > 필리핀 > 캐나다 > 캄보디아 > 스리랑카 > 미얀마 > 일본 > 몽골 ...

In [ ]:
frgn['nationality'].value_counts().plot.pie(autopct = "%.1f%%",  startangle = 45, explode = (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8))

# EDA
- notation : 국적 별로 패턴이 매우 상이하기 때문에 외국인 장기 체류 안에서 살펴볼 경우 국적 별 유형화가 시급하다.

## 전체기간(일단위) resd / work / visit population

In [ ]:
frgn.groupby(['base_date'])['resd_pop_cnt','work_pop_cnt','visit_pop_cnt'].sum().reset_index().set_index('base_date').corr()

### 전체 기간동안 거주지역에 따른 유형별 유동인구 추이
- 방문인구의 경우 그외 거주 유동인구의 영향이 매우 큼(국내 유동인구보다 심하다)
- 거주인구 > 방문인구 > 근무인구 : 방문인구의 정의가 거주지 혹은 근무지 이외의 유동인구이므로 **방문인구의 유형화**가 필요할 것으로 보임(관광지, 상권지역 등) => 방문인구는 관광하는 유동인구를 포함한 것이 기정사실.
- 거주인구와 방문인구는 음의 상관관계
- 거주지역과 그외 지역의 구분이 명확 : 거주지역에는 방문인구가 잡히지만, 그외지역에는 거주인구가 아예 잡히지 않음. => 단순히 거주지역으로의 방문객?

In [ ]:
# 거주 지역 상관 없이
pop_cnt = frgn.groupby(['base_date'])['resd_pop_cnt','work_pop_cnt','visit_pop_cnt'].sum().reset_index()
pop_cnt = pd.melt(pop_cnt, id_vars='base_date', var_name='pop', value_name='cnt')

sns.lineplot(data=pop_cnt,x='base_date',y='cnt',hue='pop')
plt.title('전 지역 population cnt')
plt.tight_layout()
plt.show()

In [ ]:
# 제주 거주
res_cnt = frgn[frgn['resd'] == '제주'].groupby(['base_date'])['resd_pop_cnt','work_pop_cnt','visit_pop_cnt'].sum().reset_index()
res_cnt = pd.melt(res_cnt, id_vars='base_date', var_name='pop', value_name='cnt')
res_cnt['base_date'] = pd.to_datetime(res_cnt['base_date'])

sns.lineplot(data=res_cnt,x='base_date',y='cnt',hue='pop')
plt.title('거주지역 제주 population cnt')
plt.show()

In [ ]:
# 그 외 거주
etc_cnt = frgn[frgn['resd'] == '그외'].groupby(['base_date'])['resd_pop_cnt','work_pop_cnt','visit_pop_cnt'].sum().reset_index()
etc_cnt = pd.melt(etc_cnt, id_vars='base_date', var_name='pop', value_name='cnt')
etc_cnt['base_date'] = pd.to_datetime(etc_cnt['base_date'])

sns.lineplot(data=etc_cnt,x='base_date',y='cnt',hue='pop')
plt.title('거주지역 그 외 population cnt')
plt.show()

### 전체 기간동안 국적별 유동인구 추이
- 국내 유동인구와 달리 국적에 해당하는 유동인구가 많은 순으로 봤을 때, 해당 국적의 유동인구가 많다고 어느 한 패턴으로 유지되지는 않는 듯.
- 국적 별로 유동인구의 패턴과 추이가 상이함 => 거주/근무/방문인구가 우세한지에 따라 국적(나라)들을 패턴화 할 수 있지 않을까? => 관광(방문)이 많은 나라, 거주인구가 많은 나라 등...

In [ ]:
# 거주지 상관없이 국적별 전체 유동인구 추이
for nation in nation_sort:
    pop_cnt = frgn[frgn['nationality'] == nation].groupby(['base_date'])['resd_pop_cnt','work_pop_cnt','visit_pop_cnt'].sum().reset_index()
    pop_cnt = pd.melt(pop_cnt, id_vars='base_date', var_name='pop', value_name='cnt')

    sns.lineplot(data=pop_cnt,x='base_date',y='cnt',hue='pop')
    plt.title('전 지역 국적 : {} population cnt'.format(nation))
    plt.tight_layout()
    plt.show()

In [ ]:
# 제주 거주 국적별 전체 유동인구 추이
for nation in nation_sort:
    pop_cnt = frgn[(frgn['nationality'] == nation) & (frgn['resd'] == '제주')].groupby(['base_date'])['resd_pop_cnt','work_pop_cnt','visit_pop_cnt'].sum().reset_index()
    pop_cnt = pd.melt(pop_cnt, id_vars='base_date', var_name='pop', value_name='cnt')

    sns.lineplot(data=pop_cnt,x='base_date',y='cnt',hue='pop')
    plt.title('제주 거주 국적 : {} population cnt'.format(nation))
    plt.tight_layout()
    plt.show()

In [ ]:
# 그외 거주 국적별 전체 유동인구 추이 => 제주에 거주하지 않으므로 resd_pop_cnt는 포함하지 않음
for nation in nation_sort:
    pop_cnt = frgn[(frgn['nationality'] == nation) & (frgn['resd'] == '그외')].groupby(['base_date'])['work_pop_cnt','visit_pop_cnt'].sum().reset_index()
    pop_cnt = pd.melt(pop_cnt, id_vars='base_date', var_name='pop', value_name='cnt')

    sns.lineplot(data=pop_cnt,x='base_date',y='cnt',hue='pop')
    plt.title('그외 거주 국적 : {} population cnt'.format(nation))
    plt.tight_layout()
    plt.show()

### 일평균 인구통계 특성별 유동인구 추이

In [ ]:
# 일평균 국적별 유동인구(중국)
nat_pop = frgn[frgn['nationality'] == 'CHN'].groupby(['base_date','nationality','resd'])['visit_pop_cnt','resd_pop_cnt','work_pop_cnt'].sum().reset_index()
nat_pop = (nat_pop.groupby(['nationality','resd'])['visit_pop_cnt','resd_pop_cnt','work_pop_cnt'].mean()).stack().reset_index()
nat_pop.columns = ['nationality','resd','pop','cnt']

figure, (ax1,ax2) = plt.subplots(ncols=2)
figure.set_size_inches(18,7)
sns.barplot(data = nat_pop[nat_pop['resd'] == '제주'].reset_index(drop=True),x = 'cnt',y='nationality',hue='pop',ax = ax1)
sns.barplot(data = nat_pop[nat_pop['resd'] == '그외'].reset_index(drop=True),x = 'cnt',y='nationality',hue='pop',ax = ax2)
ax1.set(title="거주지역이 제주인 중국 일평균 유동인구")
ax2.set(title="거주지역이 그외인 중국 일평균 유동인구")

In [ ]:
# 일평균 국적별 유동인구(중국 제외)
day_nat_pop = frgn[frgn['nationality'] != 'CHN'].groupby(['base_date','nationality','resd'])['visit_pop_cnt','resd_pop_cnt','work_pop_cnt'].sum().reset_index()
day_nat_pop = (day_nat_pop.groupby(['nationality','resd'])['visit_pop_cnt','resd_pop_cnt','work_pop_cnt'].mean()).stack().reset_index()
day_nat_pop.columns = ['nationality','resd','pop','cnt']

figure, (ax1,ax2) = plt.subplots(ncols=2)
figure.set_size_inches(18,20)
sns.barplot(data = day_nat_pop[day_nat_pop['resd'] == '제주'].reset_index(drop=True),x = 'cnt',y='nationality',hue='pop',ax = ax1, orient='h')
sns.barplot(data = day_nat_pop[day_nat_pop['resd'] == '그외'].reset_index(drop=True),x = 'cnt',y='nationality',hue='pop',ax = ax2, orient='h')
ax1.set(title="거주지역이 제주인 국적별 일평균 유동인구")
ax2.set(title="거주지역이 그외인 국적별 일평균 유동인구")

## 계절별 resd / work / visit popualation

### 국적별 유동인구

In [ ]:
season_cnt = frgn.groupby(['nationality','season'])['resd_pop_cnt','work_pop_cnt','visit_pop_cnt'].sum().stack().reset_index()
season_cnt.columns = ['nationality','season','pop','cnt']

In [ ]:
for nat in frgn.nationality.unique():
    season = season_cnt[season_cnt['nationality'] == nat]
    sns.barplot(data = season,x = 'pop',y='cnt',hue='season')
    plt.title('{} 계절별 방문인구'.format(nat))
    plt.show()

## 월별 resd / work / visit popualation 

### 국적별 유동인구

In [ ]:
month_cnt = frgn.groupby(['nationality','month'])['resd_pop_cnt','work_pop_cnt','visit_pop_cnt'].sum().stack().reset_index()
month_cnt.columns = ['nationality','month','pop','cnt']

In [ ]:
for nat in frgn.nationality.unique():
    monthly = month_cnt[month_cnt['nationality'] == nat]
    sns.lineplot(data = monthly,x = 'month',y='cnt',hue='pop')
    plt.title('{} 월별 방문인구'.format(nat))
    plt.show()

## 시간대별 resd / work / visit population

[인구 성격 별 시간당 유동인구 추이]
- 근무인구보다 방문인구 쪽이 더 늦은 시간에 인구가 줄어든다 => 방문인구에 출퇴근 인구가 포함되어있을 가능성
- 같은 visit_population이라도 지역 별로 시간당 인구 추이가 다름 => 해당 지역의 인구통계를 연관지어 생각할 필요 있음

### 시간 당 계절 별 유동인구 추이

resd_pop_cnt

In [ ]:
# 거주지역이 제주인 resd_pop_cnt = 전체 지역 resd_pop_cnt
m = frgn.groupby(['base_date','season','time'])['resd_pop_cnt'].sum().reset_index()
m = m.groupby(['season','time'])['resd_pop_cnt'].mean().reset_index()
m['time'] = m['time'].astype('category')
#m['month'] = m['month'].astype('category')
sns.lineplot(data=m,x='time',y='resd_pop_cnt',hue='season')
plt.title('시간별 계절 평균 resd_pop_cnt')

In [ ]:
# 시간별 계절 평균 resd_pop_cnt(그외 거주)는 관측 데이터 없음.

work_pop_cnt

In [ ]:
m = frgn.groupby(['base_date','season','time'])['work_pop_cnt'].sum().reset_index()
m = m.groupby(['season','time'])['work_pop_cnt'].mean().reset_index()
m['time'] = m['time'].astype('category')
sns.lineplot(data=m,x='time',y='work_pop_cnt',hue='season')
plt.title('시간별 계절 평균 work_pop_cnt')

In [ ]:
m = frgn[frgn['resd'] == '제주'].groupby(['base_date','season','time'])['work_pop_cnt'].sum().reset_index()
m = m.groupby(['season','time'])['work_pop_cnt'].mean().reset_index()
m['time'] = m['time'].astype('category')
#m['month'] = m['month'].astype('category')
sns.lineplot(data=m,x='time',y='work_pop_cnt',hue='season')
plt.title('시간별 계절 평균 work_pop_cnt(제주 거주)')

In [ ]:
m = frgn[frgn['resd'] == '그외'].groupby(['base_date','season','time'])['work_pop_cnt'].sum().reset_index()
m = m.groupby(['season','time'])['work_pop_cnt'].mean().reset_index()
m['time'] = m['time'].astype('category')
#m['month'] = m['month'].astype('category')
sns.lineplot(data=m,x='time',y='work_pop_cnt',hue='season')
plt.title('시간별 계절 평균 work_pop_cnt(그외 거주)')

visit_pop_cnt

In [ ]:
m = frgn.groupby(['base_date','season','time'])['visit_pop_cnt'].sum().reset_index()
m = m.groupby(['season','time'])['visit_pop_cnt'].mean().reset_index()
m['time'] = m['time'].astype('category')
#m['month'] = m['month'].astype('category')
sns.lineplot(data=m,x='time',y='visit_pop_cnt',hue='season')
plt.title('시간별 계절 평균 visit_pop_cnt')

In [ ]:
m = frgn[frgn['resd'] == '제주'].groupby(['base_date','season','time'])['visit_pop_cnt'].sum().reset_index()
m = m.groupby(['season','time'])['visit_pop_cnt'].mean().reset_index()
m['time'] = m['time'].astype('category')
#m['month'] = m['month'].astype('category')
sns.lineplot(data=m,x='time',y='visit_pop_cnt',hue='season')
plt.title('시간별 계절 평균 visit_pop_cnt(제주 거주)')

In [ ]:
m = frgn[frgn['resd'] == '그외'].groupby(['base_date','season','time'])['visit_pop_cnt'].sum().reset_index()
m = m.groupby(['season','time'])['visit_pop_cnt'].mean().reset_index()
m['time'] = m['time'].astype('category')
#m['month'] = m['month'].astype('category')
sns.lineplot(data=m,x='time',y='visit_pop_cnt',hue='season')
plt.title('시간별 계절 평균 visit_pop_cnt(그외 거주)')

### 시간 당 국적 별 유동인구 추이

In [ ]:
m = frgn.groupby(['base_date','nationality','resd','time'])['resd_pop_cnt','visit_pop_cnt','work_pop_cnt'].sum().reset_index()
m = m.groupby(['nationality','resd','time'])['resd_pop_cnt','visit_pop_cnt','work_pop_cnt'].mean().stack().reset_index()
m['time'] = m['time'].astype('category')
m['nationality'] = m['nationality'].astype('category')
m.columns = ['nationality','resd','time','pop','cnt']

In [ ]:
# 그외 거주의 work_pop_cnt와 resd_pop_cnt는 값이 너무 작아서 시각화에서 제외.
for nat in frgn.nationality.unique():
    figure, (ax1,ax2) = plt.subplots(ncols=2)
    figure.set_size_inches(18,7)
    sns.lineplot(data=m[(m['nationality'] == nat) & (m['resd'] == '제주')],x='time',y='cnt',hue='pop',ax=ax1)
    sns.lineplot(data=m[(m['nationality'] == nat) & (m['resd'] == '그외') & (m['pop'] == 'visit_pop_cnt')],x='time',y='cnt',hue='pop',ax=ax2)
    plt.title('{} 시간당 평균 pop_cnt(제주 거주)'.format(nat))
    ax1.set(title='{} 시간당 평균 pop_cnt(제주 거주)'.format(nat))
    ax2.set(title='{} 시간당 평균 pop_cnt(그외 거주)'.format(nat))
    plt.tight_layout()
    plt.show()

## TOP10 행정구역 선정
- 세 가지의 인구 유형 별 유동인구 TOP10 행정동을 각각 선별
- 보통 유동인구 유형별 전체 인구 TOP10을 따라가며, 인구특성별로 보면 하위권 지역은 잘 바뀌는데 상위권은 거의 고정

In [ ]:
frgn_cnt = frgn.groupby(['emd_nm'])['visit_pop_cnt','work_pop_cnt', 'resd_pop_cnt'].sum().reset_index()

# 인구 성격별 유동인구 top10 지역 추출
visit_top10 = frgn_cnt.sort_values(ascending=False,by=['visit_pop_cnt'])[:10]
work_top10 = frgn_cnt.sort_values(ascending=False,by=['work_pop_cnt'])[:10]
resd_top10 = frgn_cnt.sort_values(ascending=False,by=['resd_pop_cnt'])[:10]

In [ ]:
visit_top10.set_index('emd_nm').sort_values(by = 'visit_pop_cnt')['visit_pop_cnt'].plot(kind='barh',title = '방문인구 top10 지역', fontsize=13)

In [ ]:
work_top10.set_index('emd_nm').sort_values(by = 'work_pop_cnt')['work_pop_cnt'].plot(kind='barh',title = '근무 인구 top10 지역', fontsize=13)

In [ ]:
resd_top10.set_index('emd_nm').sort_values(by = 'resd_pop_cnt')['resd_pop_cnt'].plot(kind='barh',title = '거주인구 top10 지역', fontsize=13)

In [ ]:
nation_sort = frgn['nationality'].value_counts().index[:]
for nat in nation_sort:
    nat_cnt = frgn[frgn['nationality'] == nat].groupby(['emd_nm'])['visit_pop_cnt','work_pop_cnt', 'resd_pop_cnt'].sum().reset_index()

    # 인구 성격별 유동인구 top10 지역 추출
    visit_top10 = nat_cnt.sort_values(ascending=False,by=['visit_pop_cnt'])[:10]
    work_top10 = nat_cnt.sort_values(ascending=False,by=['work_pop_cnt'])[:10]
    resd_top10 = nat_cnt.sort_values(ascending=False,by=['resd_pop_cnt'])[:10]
    
    figure, (ax1,ax2,ax3) = plt.subplots(ncols=3)
    figure.set_size_inches(18,8)
    
    visit_top10.set_index('emd_nm').sort_values(by = 'visit_pop_cnt')['visit_pop_cnt'].plot(kind='barh',title = '방문인구 top10 지역', fontsize=13,ax=ax1)
    work_top10.set_index('emd_nm').sort_values(by = 'work_pop_cnt')['work_pop_cnt'].plot(kind='barh',title = '근무 인구 top10 지역', fontsize=13,ax=ax2)
    resd_top10.set_index('emd_nm').sort_values(by = 'resd_pop_cnt')['resd_pop_cnt'].plot(kind='barh',title = '거주인구 top10 지역', fontsize=13,ax=ax3)
    
    ax1.set(title='{} 방문인구 TOP10 지역'.format(nat))
    ax2.set(title='{} 근무인구 TOP10 지역'.format(nat))
    ax3.set(title='{} 거주인구 TOP10 지역'.format(nat))

### 전체기간 동안 유동인구 TOP10(visit, work, resd) 추이 확인

In [ ]:
V = frgn[frgn['emd_nm'].isin(visit_top10.emd_nm)].groupby(['emd_nm','base_date','resd'])['resd_pop_cnt', 'work_pop_cnt', 'visit_pop_cnt'].sum().stack().reset_index()
W = frgn[frgn['emd_nm'].isin(work_top10.emd_nm)].groupby(['emd_nm','base_date','resd'])['resd_pop_cnt', 'work_pop_cnt', 'visit_pop_cnt'].sum().stack().reset_index()
R = frgn[frgn['emd_nm'].isin(resd_top10.emd_nm)].groupby(['emd_nm','base_date','resd'])['resd_pop_cnt', 'work_pop_cnt', 'visit_pop_cnt'].sum().stack().reset_index()

V.columns = ['emd_nm','base_date','resd','pop','cnt']
W.columns = ['emd_nm','base_date','resd','pop','cnt']
R.columns = ['emd_nm','base_date','resd','pop','cnt']

Visit TOP10

In [ ]:
for nm in visit_top10['emd_nm']:
    sns.lineplot(data=V[V['emd_nm'] == nm].groupby(['emd_nm','base_date','pop'])['cnt'].sum().reset_index(),x='base_date',y='cnt',hue='pop')
    plt.title('전 거주지역 방문인구 TOP10 {} 유동인구 추이'.format(nm))
    plt.show()

Work TOP10

In [ ]:
for nm in work_top10['emd_nm']:
    sns.lineplot(data=W[W['emd_nm'] == nm].groupby(['emd_nm','base_date','pop'])['cnt'].sum().reset_index(),x='base_date',y='cnt',hue='pop')
    plt.title('전 거주지역 근무인구 TOP10 {} 유동인구 추이'.format(nm))
    plt.show()

Resd TOP10

In [ ]:
for nm in resd_top10['emd_nm']:
    sns.lineplot(data=R[R['emd_nm'] == nm].groupby(['emd_nm','base_date','pop'])['cnt'].sum().reset_index(),x='base_date',y='cnt',hue='pop')
    plt.title('전 거주지역 거주인구 TOP10 {} 유동인구 추이'.format(nm))
    plt.show()

### 시간 당 지역별 유동인구 TOP10(visit, work, resd) 추이 확인
- 내국인 유동인구 결론과 동일.
- 거주지가 제주인 경우와 아닌 경우의 시간당 유동인구 패턴이 다른 행정동이 있음
- 유동인구 세 가지 유형별 top10 지역을 다 따로 보는 것보다, 행정동마다 거주 지역 여부에 따른 시간당 유동인구 패턴을 보아야 할 듯

In [ ]:
v = frgn[frgn['emd_nm'].isin(visit_top10.emd_nm)].groupby(['emd_nm','resd','time'])['resd_pop_cnt', 'work_pop_cnt', 'visit_pop_cnt'].sum().stack().reset_index()
w = frgn[frgn['emd_nm'].isin(work_top10.emd_nm)].groupby(['emd_nm','resd','time'])['resd_pop_cnt', 'work_pop_cnt', 'visit_pop_cnt'].sum().stack().reset_index()
r = frgn[frgn['emd_nm'].isin(resd_top10.emd_nm)].groupby(['emd_nm','resd','time'])['resd_pop_cnt', 'work_pop_cnt', 'visit_pop_cnt'].sum().stack().reset_index()

v.columns = ['emd_nm','resd','time','pop','cnt']
w.columns = ['emd_nm','resd','time','pop','cnt']
r.columns = ['emd_nm','resd','time','pop','cnt']

Visit TOP10

In [ ]:
for nm in visit_top10['emd_nm']:
    figure, (ax1,ax2,ax3) = plt.subplots(ncols=3)
    figure.set_size_inches(18,5)
    sns.lineplot(data = v[(v['emd_nm'] == nm) & (v['pop'] == 'visit_pop_cnt')].reset_index(drop=True),x = 'time',y='cnt',hue='resd',ax = ax1)
    sns.lineplot(data = v[(v['emd_nm'] == nm) & (v['pop'] == 'work_pop_cnt')].reset_index(drop=True),x = 'time',y='cnt',hue='resd',ax = ax2)
    sns.lineplot(data = v[(v['emd_nm'] == nm) & (v['pop'] == 'resd_pop_cnt')].reset_index(drop=True),x = 'time',y='cnt',hue='resd',ax = ax3)
    ax1.set(title="방문인구 TOP 10 {}\n 거주지 별 시간당 유동인구(visit_pop)".format(nm))
    ax2.set(title="방문인구 TOP 10 {}\n 거주지 별 시간당 유동인구(work_pop)".format(nm))
    ax3.set(title="방문인구 TOP 10 {}\n 거주지 별 시간당 유동인구(resd_pop)".format(nm))

Work TOP10

In [ ]:
for nm in work_top10['emd_nm']:
    figure, (ax1,ax2,ax3) = plt.subplots(ncols=3)
    figure.set_size_inches(18,5)
    sns.lineplot(data = w[(w['emd_nm'] == nm) & (w['pop'] == 'visit_pop_cnt')].reset_index(drop=True),x = 'time',y='cnt',hue='resd',ax = ax1)
    sns.lineplot(data = w[(w['emd_nm'] == nm) & (w['pop'] == 'work_pop_cnt')].reset_index(drop=True),x = 'time',y='cnt',hue='resd',ax = ax2)
    sns.lineplot(data = w[(w['emd_nm'] == nm) & (w['pop'] == 'resd_pop_cnt')].reset_index(drop=True),x = 'time',y='cnt',hue='resd',ax = ax3)
    ax1.set(title="근무인구 TOP 10 {}\n 거주지 별 시간당 유동인구(visit_pop)".format(nm))
    ax2.set(title="근무인구 TOP 10 {}\n 거주지 별 시간당 유동인구(work_pop)".format(nm))
    ax3.set(title="근무인구 TOP 10 {}\n 거주지 별 시간당 유동인구(resd_pop)".format(nm))

Resd TOP10

In [ ]:
for nm in resd_top10['emd_nm']:
    figure, (ax1,ax2,ax3) = plt.subplots(ncols=3)
    figure.set_size_inches(18,5)
    sns.lineplot(data = r[(r['emd_nm'] == nm) & (r['pop'] == 'visit_pop_cnt')].reset_index(drop=True),x = 'time',y='cnt',hue='resd',ax = ax1)
    sns.lineplot(data = r[(r['emd_nm'] == nm) & (r['pop'] == 'work_pop_cnt')].reset_index(drop=True),x = 'time',y='cnt',hue='resd',ax = ax2)
    sns.lineplot(data = r[(r['emd_nm'] == nm) & (r['pop'] == 'resd_pop_cnt')].reset_index(drop=True),x = 'time',y='cnt',hue='resd',ax = ax3)
    ax1.set(title="거주인구 TOP 10 {}\n 거주지 별 시간당 유동인구(visit_pop)".format(nm))
    ax2.set(title="거주인구 TOP 10 {}\n 거주지 별 시간당 유동인구(work_pop)".format(nm))
    ax3.set(title="거주인구 TOP 10 {}\n 거주지 별 시간당 유동인구(resd_pop)".format(nm))